In [39]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [10]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-10-29 04:02:28--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   112MB/s    in 0.2s    

2024-10-29 04:02:29 (112 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [99]:
# import csv data into dataframes
books = pd.read_csv("BX-Books.csv", sep=';', encoding='ISO-8859-1', on_bad_lines='skip', low_memory=False)
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=';', encoding='ISO-8859-1')
users = pd.read_csv("BX-Users.csv", sep=';', encoding='ISO-8859-1')

In [100]:
user_counts = ratings['User-ID'].value_counts()
book_counts = ratings['ISBN'].value_counts()
filtered_users = user_counts[user_counts >= 200].index
filtered_books = book_counts[book_counts >= 100].index
ratings = ratings[(ratings['User-ID'].isin(filtered_users)) & (ratings['ISBN'].isin(filtered_books))]

# Create a pivot table
book_user_matrix = ratings.pivot(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)
book_user_sparse_matrix = csr_matrix(book_user_matrix)

# Fit the KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(book_user_sparse_matrix)

isbn_to_title = dict(zip(books['ISBN'], books['Book-Title']))
title_to_isbn = {v: k for k, v in isbn_to_title.items()}

In [103]:
# function to return recommended books - this will be tested
def get_recommends(book_title = ""):
    if book_title not in title_to_isbn:
        return f"'{book_title}' not found in the dataset."

    book_id = title_to_isbn[book_title]
    book_idx = book_user_matrix.index.get_loc(book_id)

    distances, indices = model_knn.kneighbors(book_user_sparse_matrix[book_idx], n_neighbors=6)

    similar_books = []
    for i in range(1, len(distances.flatten())):
        similar_book_isbn = book_user_matrix.index[indices.flatten()[i]]
        similar_book_title = isbn_to_title[similar_book_isbn]
        similar_books.append([similar_book_title, distances.flatten()[i]])

    return [book_title, similar_books[1:6]]

In [105]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', 0.7677075092617776], ['The Surgeon', 0.7699410973804288], ['The Weight of Water', 0.7708583572697412], ["I'll Be Seeing You", 0.8016210581447822]]]
You passed the challenge! 🎉🎉🎉🎉🎉
